In [1]:
import  pandas as pd
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("titanic")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "hadoop")
    .config("spark.sql.catalog.demo.warehouse", "s3a://bronze/warehouse/")
    .config("spark.sql.catalog.silver", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.silver.type", "hadoop")
    .config("spark.sql.catalog.silver.warehouse", "s3a://silver/warehouse/")
    .config("spark.sql.catalog.gold", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.gold.type", "hadoop")
    .config("spark.sql.catalog.gold.warehouse", "s3a://gold/warehouse/")        
    .config("spark.hadoop.fs.s3a.access.key", "teste")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

%load_ext 00-prettytables

your 131072x1 screen size is bogus. expect trouble
25/10/12 07:05:45 WARN Utils: Your hostname, DESKTOP-4LPUPP7 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/12 07:05:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/andre/miniforge3/envs/iceberg/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andre/.ivy2/cache
The jars for the packages stored in: /home/andre/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5ecdec31-ee84-4245-8852-c45efff65c5c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 190ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	-------------------

✅ Spark SQL Magic carregado! Use %sql ou %%sql para executar consultas Spark SQL.


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv")


In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS demo.ml_titanic_raw.titanic
""")

print("✅ Tabela criada com sucesso!")

25/10/11 15:29:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


✅ Tabela criada com sucesso!


In [6]:
spark_df = spark.createDataFrame(df)
spark_df.write.mode("overwrite").saveAsTable("demo.ml_titanic_raw.titanic")

In [7]:
%%sql
select * from demo.ml_titanic_raw.titanic limit 3

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S


In [8]:
#%%sql 

#DROP TABLE IF EXISTS demo.ml_titanic_raw.titanic PURGE

In [9]:
%%sql
select sex, count(*) from demo.ml_titanic_raw.titanic group by sex

sex,count(1)
female,314
male,577


In [10]:
from pyspark.sql import functions as F

# 1️⃣ Ler dados da camada bronze
df_raw = spark.table("demo.ml_titanic_raw.titanic")

# 2️⃣ Padronizar nomes das colunas
for old_col in df_raw.columns:
    new_col = old_col.strip().lower().replace(" ", "_")
    df_raw = df_raw.withColumnRenamed(old_col, new_col)

# 3️⃣ Normalizar strings
df_clean = (
    df_raw
    .withColumn("name", F.trim(F.col("name")))
    .withColumn("sex", F.lower(F.trim(F.col("sex"))))
    .withColumn("embarked", F.lower(F.trim(F.col("embarked"))))
)

# 4️⃣ Corrigir NaN (convertendo-os para null)
for c in ["age", "fare"]:
    df_clean = df_clean.withColumn(c, F.when(F.isnan(F.col(c)) | F.isnull(F.col(c)), None).otherwise(F.col(c)))

# 5️⃣ Substituir valores nulos
avg_age = df_clean.select(F.round(F.avg("age"), 1).alias("avg_age")).collect()[0]["avg_age"]
df_clean = df_clean.fillna({"age": avg_age, "embarked": "s"})

# 6️⃣ Criar coluna derivada
df_clean = df_clean.withColumn("is_child", F.when(F.col("age") < 18, 1).otherwise(0))

# 7️⃣ Padronizar sexo (male/female → 1/0)
df_clean = df_clean.withColumn("sex", F.when(F.col("sex") == "male", 1).otherwise(0))

# 8️⃣ Corrigir NaN string (como cabin)
df_clean = df_clean.withColumn(
    "cabin",
    F.when(
        (F.col("cabin").isNull()) | (F.col("cabin") == "NaN") | (F.col("cabin") == ""), 
        F.lit("-1")
    ).otherwise(F.col("cabin"))
)

# 9️⃣ Remover duplicados
df_clean = df_clean.dropDuplicates()

In [11]:
df_clean.show(10)

+-----------+--------+------+--------------------+---+----+-----+-----+-------------+-------+-----+--------+--------+
|passengerid|survived|pclass|                name|sex| age|sibsp|parch|       ticket|   fare|cabin|embarked|is_child|
+-----------+--------+------+--------------------+---+----+-----+-----+-------------+-------+-----+--------+--------+
|         91|       0|     3|Christmann, Mr. Emil|  1|29.0|    0|    0|       343276|   8.05|   -1|       s|       0|
|        264|       0|     1|Harrison, Mr. Wil...|  1|40.0|    0|    0|       112059|    0.0|  B94|       s|       0|
|         26|       1|     3|Asplund, Mrs. Car...|  0|38.0|    1|    5|       347077|31.3875|   -1|       s|       0|
|         43|       0|     3| Kraeff, Mr. Theodor|  1|29.7|    0|    0|       349253| 7.8958|   -1|       c|       0|
|        115|       0|     3|Attalah, Miss Malake|  0|17.0|    0|    0|         2627|14.4583|   -1|       c|       1|
|         99|       1|     2|Doling, Mrs. John...|  0|34

In [12]:
#df_clean.write.mode("overwrite").saveAsTable("silver.ml_titanic_silver.titanic")
df_clean.writeTo("silver.ml_titanic_silver.titanic") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()


In [13]:
%%sql
select count(*) from silver.ml_titanic_silver.titanic

count(1)
891


In [14]:
%%sql
select * from silver.ml_titanic_silver.titanic.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-10-11 11:43:38.489000,8697453100535808669,None,append,s3a://silver/warehouse/ml_titanic_silver/titanic/metadata/snap-8697453100535808669-1-e9e1df78-df64-4c37-bd62-99da6bdb8b9c.avro,"{'engine-version': '3.5.1', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1760193804119', 'added-records': '891', 'total-records': '891', 'spark.app.id': 'local-1760193804119', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '26118', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.1 (commit 8e9d59d299be42b0bca9461457cd1e95dbaad086)', 'total-files-size': '26118', 'total-data-files': '1'}"
2025-10-11 11:44:56.993000,6027476777183118668,None,append,s3a://silver/warehouse/ml_titanic_silver/titanic/metadata/snap-6027476777183118668-1-8a1e9e36-acac-4cff-abb5-7557deaf1821.avro,"{'engine-version': '3.5.1', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1760193804119', 'added-records': '891', 'total-records': '891', 'spark.app.id': 'local-1760193804119', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '26118', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.1 (commit 8e9d59d299be42b0bca9461457cd1e95dbaad086)', 'total-files-size': '26118', 'total-data-files': '1'}"
2025-10-11 11:46:28.893000,731501262550101068,None,append,s3a://silver/warehouse/ml_titanic_silver/titanic/metadata/snap-731501262550101068-1-b399bc14-0714-4b8b-a93b-fe2acb04231a.avro,"{'engine-version': '3.5.1', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1760193804119', 'added-records': '891', 'total-records': '891', 'spark.app.id': 'local-1760193804119', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '26118', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.1 (commit 8e9d59d299be42b0bca9461457cd1e95dbaad086)', 'total-files-size': '26118', 'total-data-files': '1'}"
2025-10-11 15:30:00.391000,6329781123246014202,None,append,s3a://silver/warehouse/ml_titanic_silver/titanic/metadata/snap-6329781123246014202-1-cc4fff7c-aa68-43e9-a18f-5da2aefaaf03.avro,"{'engine-version': '3.5.1', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1760207388180', 'added-records': '891', 'total-records': '891', 'spark.app.id': 'local-1760207388180', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '26118', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.6.1 (commit 8e9d59d299be42b0bca9461457cd1e95dbaad086)', 'total-files-size': '26118', 'total-data-files': '1'}"


In [15]:
# pipeline gold

In [19]:
from pyspark.sql import functions as F

# Ler dados da camada silver
df_silver = spark.table("silver.ml_titanic_silver.titanic")

# Selecionar e transformar as colunas relevantes para ML
df_ml = (
    df_silver
    # Escolher colunas úteis
    .select(
        "passengerid",
        "survived",
        "pclass",
        "sex",
        "age",
        "sibsp",
        "parch",
        "fare",
        "embarked",
        "is_child"
    )
    # Codificar valores categóricos
    .withColumn("embarked_s", F.when(F.col("embarked") == "s", 1).otherwise(0))
    .withColumn("embarked_c", F.when(F.col("embarked") == "c", 1).otherwise(0))
    .withColumn("embarked_q", F.when(F.col("embarked") == "q", 1).otherwise(0))
    # Garantir que não há nulos
    .na.fill(0)
)

In [20]:
# Criar tabela ML-ready (substitui se já existir)
df_ml.writeTo("gold.ml_titanic.titanic_features") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()

In [23]:
%%sql
select * from gold.ml_titanic.titanic_features limit 3

passengerid,survived,pclass,sex,age,sibsp,parch,fare,embarked,is_child,embarked_s,embarked_c,embarked_q
91,0,3,1,29.0,0,0,8.05,s,0,1,0,0
264,0,1,1,40.0,0,0,0.0,s,0,1,0,0
26,1,3,0,38.0,1,5,31.3875,s,0,1,0,0
